## QA-Loader

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

In [2]:
from textlong.knowledge import LocalFilesLoader, QAExcelsLoader

In [3]:
qa = QAExcelsLoader()
print(qa.help())
print(qa.get_files())

支持为 <public> 从 [/Users/xuehongwei/github/textlong/notes/data/public/__QA__] 中加载类型为 xlsx,xls 的文件。
['/Users/xuehongwei/github/textlong/notes/data/public/__QA__/消防50条问题(240520).xlsx']


In [4]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()

from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

docs = qa.load()

db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-book-u1j6yauo-py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from textlong.qa import format_qa_docs
retriever.invoke("商场扶梯下面要设置喷水吗？")

[Document(page_content='商场\n场景：商场店铺内钢楼梯下方（非扶梯）\n问题：需要设置喷头吗？', metadata={'answer': '问题答案：需要设置喷头。\n\n相关规范解释：根据《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001），商场店铺内的钢楼梯下方应设置自动喷水灭火系统的喷头，以确保在火灾发生时能够迅速有效地控制火势。钢楼梯下方可能存在电线、管道等易燃物品，且火灾蔓延速度快，因此需要喷头覆盖以增强防火能力。', 'source': '/Users/xuehongwei/github/textlong/notes/data/public/__QA__/消防50条问题(240520).xlsx', 'sheet': 'QA_50个问题'}),
 Document(page_content='商场\n场景：商场自动扶梯下方\n问题：需要设置自动喷淋灭火系统吗？', metadata={'answer': '问题答案：通常不需要。\n\n相关规范解释：根据《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001），自动扶梯下方并不被视为通常需要设置自动喷淋系统的区域。自动扶梯下方的空间一般较小，且不是主要的火灾荷载区域。然而，如果自动扶梯附近有其他火灾风险较高的区域或设施，如电气室、机械设备等，则应按照规范的相关要求考虑设置自动喷淋系统。', 'source': '/Users/xuehongwei/github/textlong/notes/data/public/__QA__/消防50条问题(240520).xlsx', 'sheet': 'QA_50个问题'}),
 Document(page_content='高层建筑 \n场景：高层住宅的裙房内的网吧\n问题：需要设置自动喷淋系统吗？', metadata={'answer': '问题答案：需要设喷淋。\n\n相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.4.9条规定，高层建筑内的歌舞娱乐放映游艺场所（如网吧）应设置自动喷水灭火系统。此外，根据第3.1.1条，网吧属于人员密集场所，且具有较高的火灾风险，因此需要采取更为严格的防火措施，包括设

In [5]:
from langchain_zhipu import ChatZhipuAI
from textlong.memory import MemoryManager, WithMemoryBinding
from textlong.qa import format_qa_docs, create_qa_prompt
from textlong.hub import load_chat_prompt

# prompt = create_qa_prompt()
prompt = load_chat_prompt(template_id="qa", project_id="xiaofang")
llm = ChatZhipuAI()
retriever = db.as_retriever()

chain = {
    "context":  (lambda x: x['input']) | retriever | format_qa_docs,
    "question": lambda x: x['input'],
    "history":  lambda x: x['history'],
} | prompt | llm

memory = MemoryManager()
withMemoryChain = WithMemoryBinding(chain, memory)
config = {"configurable": {"session_id": "1"}}

In [6]:
# chain = create_qa_chain(llm, retriever)
for x in withMemoryChain.stream({"input": "商场扶梯下面要设置喷水吗？"}, config):
    print(x.content, end="|")

商场|扶|梯|下面|通常|不需要|设置|喷|水|系统|。|因为|扶|梯|下方|空间|较小|，|不是|主要的|火灾|荷载|区域|，|且|扶|梯|的|金属材料|不易|燃烧|。|但|若|扶|梯|附近|有|其他|火灾|风险|较高的|区域|或|设施|，|应|按|规范|要求|考虑|是否|设置|喷|水|系统|。||

In [7]:
# chain = create_qa_chain(llm, retriever)
for x in withMemoryChain.stream({"input": "下面有储物间呢？"}, config):
    print(x.content, end="|")

如果|商场|扶|梯|下面|有|储|物|间|，|那么|根据|储|物|间|内|储存|物品|的|火灾|风险|以及|空间|大小|，|可能|需要|设置|喷|水|灭火|系统|。|一般|而言|，|如果|储|物|间|内|储存|的是|易|燃|物品|，|或者|储|物|间|面积|较大|，|按照|相关|防火|规范|，|应该|设置|自动|喷|水|灭火|系统|。|这|旨在|提高|火灾|时的|灭火|效率和|安全性|。|具体|是否|需要|设置|，|应|参照|当地的|建筑设计|防火|规范|和|自动|喷|水|灭火|系统|设计|规范|。||

In [8]:
create_qa_prompt()

ChatPromptTemplate(input_variables=['context', 'history', 'question'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'task_instruction': '\n你是一名咨询专家，只负责根据资料回答相关提问，禁止回答与此无关的问题。\n\n1. 如果你获得的参考例子无法回答问题，可以查询互联网，但务必注意资料的真实性，不要做任何编造\n2. 请使用简洁的语言回答，不要啰嗦\n3. 不要生成"根据提供的资料..."等字眼\n', 'output_format': '\n输出样例：\n```\n问题答案：xxx。\n\n相关规范解释：xxxxxxxx\n```\n'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['task_instruction'], template='{{task_instruction}}', template_format='mustache')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我有哪些资料可以参考？', template_format='mustache')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'output_f

In [9]:
load_chat_prompt("qa", "消防", in_memory=False)

ChatPromptTemplate(input_variables=['context', 'history', 'question'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'output_format': '\n输出样例：\n```\n问题答案：xxx。\n\n相关规范解释：xxxxxxxx\n```\n', 'task_instruction': '\n你是一名咨询专家，只负责根据资料回答相关提问，禁止回答与此无关的问题。\n\n1. 如果你获得的参考例子无法回答问题，可以查询互联网，但务必注意资料的真实性，不要做任何编造\n2. 请使用简洁的语言回答，不要啰嗦\n3. 不要生成"根据提供的资料..."等字眼\n'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['task_instruction'], template='{{task_instruction}}', template_format='mustache')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我有哪些资料可以参考？', template_format='mustache')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'output_f

In [ ]:
from textlong.prompts import save_chat_prompt, load_chat_prompt
save_chat_prompt(prompt, "qa", "xiaofang")

In [ ]:
query = "有什么例子"
results = db.similarity_search(query)
print(results)
# for doc, score in results:
#     print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}") 

In [ ]:
%pip install faiss-cpu

In [ ]:
from textlong.prompts import save_chat_prompt, load_chat_prompt, create_writing_todo_prompt
save_chat_prompt(create_writing_todo_prompt(), "todo", "xiaofang")

In [ ]:
load_chat_prompt("todo", "xiaofang")

## sentence_transformers

In [3]:
%pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 148.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 578.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 615.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 6.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 2.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 9.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 18.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 1.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.8/415.8 kB 2.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.9 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━

## 完整代码

In [13]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()

from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 加载
from textlong import format_qa_docs, QAExcelsLoader
docs = QAExcelsLoader().load()

# 入库
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

# LLM
from langchain_zhipu import ChatZhipuAI
llm = ChatZhipuAI()

# 提示语
from textlong.prompts import create_qa_prompt, load_chat_prompt
prompt = load_chat_prompt("qa", "xiaofang")

# chain
chain = {
    "context":  (lambda x: x['input']) | retriever | format_qa_docs,
    "question": lambda x: x['input'],
    "history":  lambda x: x['history'],
} | prompt | llm

# 记忆
from textlong.memory import MemoryManager, WithMemoryBinding
memory = MemoryManager()
withMemoryChain = WithMemoryBinding(chain, memory)

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-book-u1j6yauo-py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
# 提问
config = {"configurable": {"session_id": "1"}}
for x in withMemoryChain.stream({"input": "宿舍的过道需要吗？"}, config):
    print(x.content, end="|")

需要|根据|具体情况|。

规范|解释|：|宿舍|过|道|是否|需要|设置|喷|水|系统|取决于|多种|因素|，|如|宿舍|的建筑|高度|、|人员|密集|程度|、|火灾|风险|等|。|根据|《|建筑设计|防火|规范|》，|如果|宿舍|过|道|属于|人员|密集|区域|或|存在|较高的|火灾|风险|，|应|考虑|设置|喷|水|系统|。|对于|多层|或|高层|宿舍|，|为了|提高|防火|安全性|，|过|道|通常会|设置|喷|淋|系统|。|但对于|低|层|宿舍|，|如果|过|道|宽度|适中|、|通风|良好|且|火灾|风险|较低|，|可能|不需要|设置|喷|水|系统|。||

## Tools

In [17]:
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_zhipu import ChatZhipuAI
from textlong.qa import AskDocumentTool, create_qa_chain

docs = [Document(page_content="textlong是一个长文生成的python模块。")]

llm = ChatZhipuAI()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

chain = create_qa_chain(llm, db.as_retriever())

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-book-u1j6yauo-py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
chain.invoke({"query": "textlong是什么?"})

AIMessage(content='textlong是一个长文生成的Python模块。通过这个模块，用户可以生成各种样式的长篇文章，适用于需要大量文本内容的不同场景，比如数据填充、测试文档等。', response_metadata={'id': '8689076627888601317', 'created': 1716713414, 'token_usage': {'completion_tokens': 40, 'prompt_tokens': 45, 'total_tokens': 85}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-e58ad3d1-164c-40dc-96cc-ef1b96f957a1-0')

In [21]:
tool = AskDocumentTool(chain=chain)
tool.invoke({"query": "textlong是什么?"})

AIMessage(content='textlong是一个长文生成的Python模块。通过这个模块，用户可以生成各种类型的文本内容，适用于需要大量文本数据的场景，比如文本分析、自然语言处理等领域。具体的功能和用途可能需要参考该模块的官方文档或源代码以获得更详尽的信息。', response_metadata={'id': '8689071920604292138', 'created': 1716713409, 'token_usage': {'completion_tokens': 61, 'prompt_tokens': 45, 'total_tokens': 106}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-ba845729-532e-4061-a09d-b4a1dde3b129-0')